In [3]:
import sys
import platform

import pandas as pd
import numpy as np

from utils import *

# Data Preparation
- This Step is to prepare a matrix that suitable to train the model
- The matrix is a sparse matrix which contains a lot of zero elements in the matrix
- In the matrix, each row is representing a stock code while each column is representing a shareholder
- The value of each i,j is representing how many share a shareholder holds for the particular stock

| Example | Shareholder1 | Shareholder1 |
| --- | --- | --- |
| Stock1 | shares | shares |
| Stock2 | shares | shares |
| Stock3 | shares | shares |
| ... | ... | ... |

In [8]:
# Define data_path
if platform.system() == 'Windows':
    data_path = '.\data\stock_port.csv'
else:
    data_path = './data/stock_port.csv'
    
# Load dataset from data path
df = load_data(data_path)

# Convert df to sparse matrix
sp_matrix, row_ind_dict, col_ind_dict = convert_data_sparse_matrix(df)

# List Basic Info
print('Dimension of sparse_matrix: ', sp_matrix.shape)
row_dim = sp_matrix.shape[0]
col_dim = sp_matrix.shape[1]
print('How many stock do we have in the dataset: ', row_dim)
print('How many unique shareholder do we have in the dataset', col_dim)

Dimension of sparse_matrix:  (2020, 709)
How many stock do we have in the dataset:  2020
How many unique shareholder do we have in the dataset 709


# Data Preprocessing

- This step is to normalize each column by calculating the shareholding % of each stock
- Sum over the column should result in one

In [11]:
# Calculate shareholding % by stock_code
sp_matrix_stock = sp_matrix / np.sum(sp_matrix, axis = 1).reshape(row_dim, -1)

# Sum of column should result one
np.sum(sp_matrix_stock, axis = 1)

array([1., 1., 1., ..., 1., 1., 1.])

# Data Visualization

- 
